In [1]:
import os
# data_dir = os.path.join(os.getcwd(), "drive", "MyDrive", "pneumonia")
data_dir = os.path.join("/kaggle/input/chest-xray-pneumonia/chest_xray")
print(f"Our data directory is {data_dir}")

train_dir = os.path.join(data_dir, "train")
test_dir = os.path.join(data_dir, "test")
val_dir = os.path.join(data_dir, "val")

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader


Our data directory is /kaggle/input/chest-xray-pneumonia/chest_xray


KeyboardInterrupt: 

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
traindataset = ImageFolder(root=train_dir, transform=transform)
valdataset = ImageFolder(root=val_dir, transform=transform)
train_loader = DataLoader(traindataset, batch_size=32, shuffle=True)
val_loader = DataLoader(valdataset, batch_size=32, shuffle=False)

In [3]:
import torch 
import torch.nn as nn
import torch.nn.functional as f

class Basicblock(nn.Module):
    expansion=1

    def __init__(self,in_channels,out_channels,stride=1,downsample=None):
        super(Basicblock,self).__init__()
        self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
        self.bn1=nn.BatchNorm2d(out_channels)
        self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False)
        self.bn2=nn.BatchNorm2d(out_channels)
        self.downsample= downsample
        self.relu=nn.ReLU(inplace=True)

    def forward(self,x):
        identity=x
        if self.downsample is not None:
            identity = self.downsample(x)
        out=self.conv1(x)
        out=self.bn1(out)
        out=f.relu(out)
        out=self.conv2(out)
        out=self.bn2(out)
        out+=identity
        out=f.relu(out)
        return out
class Resnet34(nn.Module):
    def __init__(self,num_classes=1):
        super(Resnet34,self).__init__()
        self.in_channels=64
        self.conv1=nn.Conv2d(3,64,kernel_size=7,stride=2,padding=3,bias=False)
        self.bn1=nn.BatchNorm2d(64)
        self.relu=nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3,stride=2,padding=1)

        self.layer1=self.makelayer(64,3)
        self.layer2=self.makelayer(128,4,stride=2)
        self.layer3=self.makelayer(256,6,stride=2)
        self.layer4=self.makelayer(512,3,stride=2)
        self.avgpool=nn.AdaptiveAvgPool2d((1,1))
        self.fc=nn.Linear(512,num_classes)
    def makelayer(self,out_channels,blocks,stride=1):
        downsample=None
        if stride!=1 or self.in_channels!=out_channels:
            downsample=nn.Sequential(nn.Conv2d(self.in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                                    nn.BatchNorm2d(out_channels))
        layers=[Basicblock(self.in_channels,out_channels,stride,downsample)]
        self.in_channels=out_channels
        for i in range(1,blocks):
            layers.append(Basicblock(self.in_channels,out_channels))
        return nn.Sequential(*layers)
    def forward(self, x):
        x=self.conv1(x)
        x=self.bn1(x)
        x=self.relu(x)
        x=self.maxpool(x)

        x=self.layer1(x)
        x=self.layer2(x)
        x=self.layer3(x)
        x=self.layer4(x)

        x=self.avgpool(x)
        x=torch.flatten(x,1)
        x=self.fc(x)
        
        return x

In [4]:
import torch.optim as optim
from torch.utils.data import DataLoader
model=Resnet34()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion=nn.BCEWithLogitsLoss()
optimizer=optim.Adam(model.parameters(),lr=1e-4)

num_epochs=10
for epoch in range(num_epochs):
    model.train()
    runningloss=0
    correct=0
    total=0

    for inputs, targets in train_loader:
        inputs=inputs.to(device)
        targets=targets.to(device).float().unsqueeze(1)
        outputs=model(inputs)
        loss=criterion(outputs,targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        runningloss+=loss.item()

        pred=torch.sigmoid(outputs)>0.5
        correct+=(pred==targets.bool()).sum().item()
        total+=targets.size(0)
    epoch_loss=runningloss/len(train_loader)
    epoch_acc=correct/total
    print(f"Epoch [{epoch+1}/{num_epochs}] | Loss: {epoch_loss:.4f} | Accuracy: {epoch_acc:.4f}")

Epoch [1/10] | Loss: 0.1759 | Accuracy: 0.9346
Epoch [2/10] | Loss: 0.0872 | Accuracy: 0.9670
Epoch [3/10] | Loss: 0.0639 | Accuracy: 0.9772
Epoch [4/10] | Loss: 0.0552 | Accuracy: 0.9797
Epoch [5/10] | Loss: 0.0367 | Accuracy: 0.9870
Epoch [6/10] | Loss: 0.0417 | Accuracy: 0.9843
Epoch [7/10] | Loss: 0.0236 | Accuracy: 0.9929
Epoch [8/10] | Loss: 0.0115 | Accuracy: 0.9964
Epoch [9/10] | Loss: 0.0203 | Accuracy: 0.9921
Epoch [10/10] | Loss: 0.0215 | Accuracy: 0.9919


In [ ]:
def print_layer_structure(model, indent=0):
    for name, layer in model.named_children():
        print("  " * indent + f"{name}: {layer.__class__.__name__}")
        print_layer_structure(layer, indent + 1)

model = ResNet34(num_classes=1)
print_layer_structure(model)